---
**License**

 TestInterp

 Mon Jan 25 20:56:00 2020\
 Copyright  2021\
 Eduardo ...\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 TestInterp\
 Copyright (C) 2021 Eduardo ...\
 Copyright (C) 2021 Sandro Dias Pinto Vitenti <vitenti@uel.br>


 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [1]:
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

import sys
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from scipy.optimize import minimize

from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
%matplotlib inline

In [2]:
__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

In [3]:
    dim = 5
    np.random.seed(seed=123)

    p = np.random.random_sample((dim,))
    print (p)

[ 0.69646919  0.28613933  0.22685145  0.55131477  0.71946897]


In [4]:
rng = Ncm.RNG.seeded_new (None, 123)

fmodel = Ncm.ModelMVND.new (dim)
fdata = Ncm.DataGaussCovMVND.new_full (dim, 0.1, 0.4, 10.0, -1.0, 1.0, rng)

fdata.props.use_norma = True

mset = Ncm.MSet.new_array ([fmodel])
mset.param_set_all_ftype (Ncm.ParamType.FREE)
mset.prepare_fparam_map ()

mset.fparams_set_array (p)
mset.pretty_log ()

#----------------------------------------------------------------------------------
# Model[00000]:
#   - NcmModelMVND : MVND
#----------------------------------------------------------------------------------
# Model parameters
#   - mu_0[00]:  0.696469185597862   [FREE]
#   - mu_1[01]:  0.286139334950379   [FREE]
#   - mu_2[02]:  0.226851453564203   [FREE]
#   - mu_3[03]:  0.551314769082891   [FREE]
#   - mu_4[04]:  0.719468969785563   [FREE]


In [5]:
y_a = []
x_a = []
xn_a = []

interp = Ncm.StatsDistNdVBKStudentt.new (len (p), Ncm.StatsDistNdCV.NONE, 3.0)
#interp = Ncm.StatsDistNdKDEGauss.new (len (p), Ncm.StatsDistNdCV.NONE)
nps = 500

for a in range (nps):
    v, N = fdata.gen (mset, None, None, rng)
    interp.add_obs (v)
    y_a.append (fdata.m2lnL_val (mset))
    x_a.append (v.dup_array ())

for a in range (nps):
    v, N = fdata.gen (mset, None, None, rng)
    xn_a.append (v.dup_array ())

y_a = np.array (y_a)
x_a = np.array (x_a)
xn_a = np.array (xn_a)

fdata.y.set_array (p)

fdata.m2lnL_val (mset)

-4.498559747474191

In [6]:
interp.set_cv_type (Ncm.StatsDistNdCV.SPLIT_FITD)
interp.prepare_interp (Ncm.Vector.new_array (y_a))
interp.set_cv_type (Ncm.StatsDistNdCV.NONE)

calib_os = interp.get_over_smooth ()

print (interp.get_rnorm (), calib_os)



COV:      0.020923    0.0057791   -0.0099936   -0.0030409   0.00081133
COV:     0.0057791    0.0084686    -0.008556    0.0018166  -0.00025798
COV:    -0.0099936    -0.008556      0.05183   -0.0015818   -0.0073048
COV:    -0.0030409    0.0018166   -0.0015818     0.019017    -0.001493
COV:    0.00081133  -0.00025798   -0.0073048    -0.001493     0.040621
COV:      0.021262    0.0029556   -0.0054887  -0.00059459    0.0060295
COV:     0.0029556    0.0086127   -0.0037728    0.0037174   -0.0028507
COV:    -0.0054887   -0.0037728     0.056121    -0.015949    0.0069678
COV:   -0.00059459    0.0037174    -0.015949     0.039064    -0.007035
COV:     0.0060295   -0.0028507    0.0069678    -0.007035     0.037024
COV:      0.038062     0.011776    -0.013181    0.0039335     0.017941
COV:      0.011776     0.012809   -0.0091107    0.0024079    0.0014006
COV:     -0.013181   -0.0091107     0.055648    0.0063993   -0.0071534
COV:     0.0039335    0.0024079    0.0063993     0.016663   -0.0020553
COV:  

COV:   -6.1018e-05     0.015771    -0.012541    0.0086566   -0.0098963
COV:     0.0072592    -0.012541      0.08889    0.0024496     0.019791
COV:    -0.0088607    0.0086566    0.0024496     0.029839    -0.025291
COV:      0.051469   -0.0098963     0.019791    -0.025291      0.12016
COV:      0.028617    0.0070235   -0.0012226    0.0037072    0.0096984
COV:     0.0070235     0.013885   -0.0047569    0.0054281    0.0012597
COV:    -0.0012226   -0.0047569     0.064255    -0.003712   -0.0036975
COV:     0.0037072    0.0054281    -0.003712     0.033506    0.0026781
COV:     0.0096984    0.0012597   -0.0036975    0.0026781     0.042577
COV:      0.031572    0.0053863    -0.003851    0.0030989     0.010984
COV:     0.0053863     0.014025   -0.0018215    0.0028651   -0.0030314
COV:     -0.003851   -0.0018215     0.042694    0.0043482   -0.0054276
COV:     0.0030989    0.0028651    0.0043482     0.023914    0.0011008
COV:      0.010984   -0.0030314   -0.0054276    0.0011008     0.042974
COV:  

COV:    -0.0059095   -0.0055729     0.099582    -0.004531   -0.0093704
COV:     -0.011302     0.010364    -0.004531      0.04826    -0.021128
COV:      0.045147   0.00014626   -0.0093704    -0.021128     0.097979
COV:      0.038101    0.0050428    -0.004187   0.00090985     0.012184
COV:     0.0050428     0.014827   -0.0023725    0.0056241   -0.0062663
COV:     -0.004187   -0.0023725     0.056354    0.0069996   -0.0015476
COV:    0.00090985    0.0056241    0.0069996      0.03247   -0.0062778
COV:      0.012184   -0.0062663   -0.0015476   -0.0062778     0.046821
COV:      0.025111    0.0023882   -0.0087284   -0.0038353    0.0021329
COV:     0.0023882     0.012971   0.00042503    0.0055423   -0.0054329
COV:    -0.0087284   0.00042503     0.046208  -0.00020418   0.00021525
COV:    -0.0038353    0.0055423  -0.00020418     0.024016   -0.0032562
COV:     0.0021329   -0.0054329   0.00021525   -0.0032562     0.036617
COV:      0.013423    0.0023478    9.789e-05    0.0008868    0.0010227
COV:  

COV:      0.031996    0.0040134   0.00017047   -0.0019822    0.0043157
COV:     0.0040134    0.0094612  -0.00018804    0.0018232   -0.0024348
COV:    0.00017047  -0.00018804      0.04822  -0.00087217    0.0025695
COV:    -0.0019822    0.0018232  -0.00087217     0.025163   -0.0041469
COV:     0.0043157   -0.0024348    0.0025695   -0.0041469     0.043228
COV:      0.025241    0.0046604   -0.0015515   -0.0021722    0.0034303
COV:     0.0046604    0.0087712    -0.005075     0.002041   -0.0012413
COV:    -0.0015515    -0.005075     0.057318   -0.0041312   -0.0056822
COV:    -0.0021722     0.002041   -0.0041312      0.02006   0.00012639
COV:     0.0034303   -0.0012413   -0.0056822   0.00012639     0.037137
COV:      0.016707    0.0010284   -0.0027146    0.0012229      0.00293
COV:     0.0010284    0.0084204   -0.0010291    0.0023317  -0.00014882
COV:    -0.0027146   -0.0010291     0.040933   -0.0037802    0.0034547
COV:     0.0012229    0.0023317   -0.0037802     0.021569   -0.0031719
COV:  

COV:      0.032837     0.005334   -0.0016986   -0.0013342    0.0053873
COV:      0.005334     0.010493    0.0044744    0.0021556  -6.8192e-05
COV:    -0.0016986    0.0044744     0.050794   0.00026412   -0.0040152
COV:    -0.0013342    0.0021556   0.00026412     0.030171    0.0042604
COV:     0.0053873  -6.8192e-05   -0.0040152    0.0042604     0.037095
COV:      0.032734    0.0087272   -0.0029222   0.00026809     0.012657
COV:     0.0087272     0.019928   -0.0059871    0.0047439   -0.0027855
COV:    -0.0029222   -0.0059871     0.054639     0.003953   -0.0052453
COV:    0.00026809    0.0047439     0.003953     0.025546    0.0033162
COV:      0.012657   -0.0027855   -0.0052453    0.0033162     0.072697
COV:      0.039911     0.012571    -0.013201   -0.0022441    0.0034813
COV:      0.012571     0.021373   -0.0030292    0.0082189   -0.0091468
COV:     -0.013201   -0.0030292     0.069209     0.013375    -0.013814
COV:    -0.0022441    0.0082189     0.013375     0.032323   -0.0064308
COV:  

...

COV:   -0.00065874    0.0092284   -0.0095776     0.035029    0.0042723
COV:      0.030919    0.0087739   -0.0071223    0.0042723     0.069833
COV:      0.030374    0.0024899    -0.004727  -0.00046434    0.0092868
COV:     0.0024899     0.011187   -0.0015153    0.0012942   -0.0034084
COV:     -0.004727   -0.0015153     0.039442   -0.0017798    0.0014691
COV:   -0.00046434    0.0012942   -0.0017798     0.017436     0.001689
COV:     0.0092868   -0.0034084    0.0014691     0.001689     0.035065
COV:      0.029911    0.0058624    0.0011816   -0.0019758     0.015548
COV:     0.0058624     0.014435   -0.0027338    0.0048848    0.0010602
COV:     0.0011816   -0.0027338     0.062715    0.0027378   -0.0085817
COV:    -0.0019758    0.0048848    0.0027378     0.033765    0.0019129
COV:      0.015548    0.0010602   -0.0085817    0.0019129     0.047751
COV:      0.021219    0.0019829   -0.0036173   -0.0074486  -0.00056066
COV:     0.0019829    0.0078414    0.0033576    0.0010019   -0.0030128
COV:  

COV:      0.033126    0.0026042    0.0021107  -0.00093554    0.0094619
COV:     0.0026042      0.01673   -0.0019076     0.006074  -0.00091137
COV:     0.0021107   -0.0019076     0.060082    0.0019039    0.0035365
COV:   -0.00093554     0.006074    0.0019039     0.025192    -0.002964
COV:     0.0094619  -0.00091137    0.0035365    -0.002964     0.046896
COV:      0.047838     0.012119   -0.0012643    -0.003618     0.012243
COV:      0.012119     0.028065    -0.022394    0.0093392   -0.0037137
COV:    -0.0012643    -0.022394     0.092397   -0.0044703     0.011904
COV:     -0.003618    0.0093392   -0.0044703     0.037722   -0.0052063
COV:      0.012243   -0.0037137     0.011904   -0.0052063       0.0659
COV:      0.030506    0.0057265   -0.0066644   -0.0039342    0.0057688
COV:     0.0057265     0.011937  -0.00048701    0.0034512   -0.0042015
COV:    -0.0066644  -0.00048701       0.0664   -0.0024644   -0.0050828
COV:    -0.0039342    0.0034512   -0.0024644     0.030235   -0.0046552
COV:  

COV:     0.0020143    0.0071117     0.067667    0.0059142   -0.0087311
COV:    -0.0073529    0.0038445    0.0059142       0.0267   -0.0021633
COV:     0.0056056     0.004167   -0.0087311   -0.0021633     0.046022
COV:      0.058198    0.0098343    0.0054264    0.0010038     0.018887
COV:     0.0098343      0.01548   -0.0013206    0.0036254   6.3147e-05
COV:     0.0054264   -0.0013206     0.064303    0.0030403    0.0011464
COV:     0.0010038    0.0036254    0.0030403     0.038921   0.00055057
COV:      0.018887   6.3147e-05    0.0011464   0.00055057     0.056129
COV:      0.013155    0.0021179    0.0012479   0.00038755    0.0073255
COV:     0.0021179     0.011373   -0.0078788    0.0015602    0.0069284
COV:     0.0012479   -0.0078788     0.031782   -0.0016559   -0.0051085
COV:    0.00038755    0.0015602   -0.0016559     0.012457   -0.0047275
COV:     0.0073255    0.0069284   -0.0051085   -0.0047275     0.040625
COV:      0.022877  -0.00015599   -0.0012579   -0.0028477    0.0087006
COV:  

COV:      0.001827    0.0038394   -0.0031893     0.015162    0.0020489
COV:      0.003949    0.0010978   -0.0036636    0.0020489     0.025702
COV:      0.048677     0.012886    0.0038079    0.0018937     0.021904
COV:      0.012886     0.017419   -0.0025593    0.0053036    0.0078041
COV:     0.0038079   -0.0025593     0.068115    0.0022744    0.0040395
COV:     0.0018937    0.0053036    0.0022744     0.028648    0.0044861
COV:      0.021904    0.0078041    0.0040395    0.0044861     0.078371
COV:      0.028456      0.00588   -0.0068047    -0.010032    0.0064343
COV:       0.00588    0.0094832    0.0016152    0.0020519   -0.0031373
COV:    -0.0068047    0.0016152     0.035209    0.0057842   -0.0066099
COV:     -0.010032    0.0020519    0.0057842     0.031483   -0.0055476
COV:     0.0064343   -0.0031373   -0.0066099   -0.0055476     0.038741
COV:      0.032701    0.0041873   -0.0073298     0.003211     0.015565
COV:     0.0041873     0.014279   0.00021558    0.0055984   -0.0021389
COV:  

COV:      0.017832    0.0079591    0.0015658   -0.0017933     0.070783
COV:      0.033139    0.0041685   4.4181e-06    0.0010732     0.013107
COV:     0.0041685     0.012091   -0.0041312     0.004661   0.00052911
COV:    4.4181e-06   -0.0041312     0.052266     0.010837    0.0044936
COV:     0.0010732     0.004661     0.010837     0.026524   -0.0012017
COV:      0.013107   0.00052911    0.0044936   -0.0012017     0.062176
COV:      0.027674    0.0059708  -0.00053706   -0.0024211    0.0068924
COV:     0.0059708     0.012039   -0.0081147     0.003052   0.00027166
COV:   -0.00053706   -0.0081147     0.057974    -0.005304  -0.00085444
COV:    -0.0024211     0.003052    -0.005304     0.022265   -0.0029925
COV:     0.0068924   0.00027166  -0.00085444   -0.0029925     0.042288
COV:      0.029835    0.0052557   -0.0077386   0.00048331    0.0038372
COV:     0.0052557    0.0099378   0.00023566    0.0013894    0.0013259
COV:    -0.0077386   0.00023566     0.052241   0.00031848   -0.0018881
COV:  

In [7]:
print (calib_os)

1.0


In [8]:
#interp.set_over_smooth (calib_os)
#interp.prepare_interp (Ncm.Vector.new_array (y_a))

In [10]:
       
yi_a = []
ya_a = []
yi_b = []

for x in xn_a:
    mset.fparams_set_array (x)
    fdata.prepare (mset)
    yi_a.append (interp.eval_m2lnp (Ncm.Vector.new_array (x)))
    ya_a.append (fdata.m2lnL_val (mset))

yi_a = np.array (yi_a)
ya_a = np.array (ya_a)

m = -np.mean (yi_a - ya_a)
diff = (yi_a + m) / ya_a - 1.0

probs = np.minimum(np.exp (-0.5*((yi_a[0::2]-ya_a[0::2])-(yi_a[1::2]-ya_a[1::2]))), 1.0)
bfact = np.abs(np.exp (-0.5*((yi_a[0::2]-ya_a[0::2])-(yi_a[1::2]-ya_a[1::2])))-1.0)

mean_err = 10**(np.mean (np.log10 (np.abs (diff))))

print ("mean error  %.2e" % (mean_err))
print ("Prob accept %.1f%%" % (100.0 * np.mean (probs)))

plt.figure (figsize=(14, 7))

plt.hist (probs, label='prob', bins = 60, density=True)
plt.grid ()
plt.legend(loc='best')
plt.show ()

plt.figure (figsize=(14, 7))

plt.hist (np.log10 (np.abs (diff)), bins = 60, label=r'$|y_i/y_a - 1|$')
plt.grid ()
plt.legend(loc='best')
plt.show ()


[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  